In [74]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import time
import pandas as pd
import re
import os

무신사 브랜드 내용가져오기

In [58]:
options = webdriver.ChromeOptions()

options.add_argument('--disable-dev-shm-usage') 
options.add_argument("--no-sandbox") 
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)


data = []
for page_num in tqdm(range(1,41), desc='pages'):
    url = f'https://www.musinsa.com/brands?categoryCode=001&type=&sortCode=BRAND_RANK&page={page_num}&size=100'
    driver.get(url)
    
    # 한 페이지의 모든 브랜드 정보를 로드할 때까지 기다림
    wait = WebDriverWait(driver, 6)
    wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="text_list"]/ul/li')))

    for i in tqdm(range(1, 101), desc='Brands', leave=False):  # tqdm으로 브랜드 진행 상황 표시
        try:
            Brand_KR = driver.find_element(By.XPATH, f'//*[@id="text_list"]/ul/li[{i}]/dl/dt/a').text
            Brand_ENG_Element = driver.find_element(By.XPATH, f'//*[@id="text_list"]/ul/li[{i}]/dl/dd/a')
            Brand_ENG = Brand_ENG_Element.text.split('(')[0].strip() 
            Count = Brand_ENG_Element.find_element(By.TAG_NAME, 'span').text 
            Count = Count.replace('(', '').replace(')', '').replace(',', '')

            data.append([Brand_KR, Brand_ENG, Count])  # 데이터 리스트에 추가
        except NoSuchElementException:
            continue
driver.quit()

# 데이터프레임 생성 및 출력
df = pd.DataFrame(data, columns=['Brand_KR', 'Brand_ENG', '개수'])

df.to_csv('BrandName.csv', index=False)



pages: 100%|██████████| 40/40 [03:37<00:00,  5.44s/it]


브랜드 이미지, 정보,  since 가져오기

In [81]:
df = pd.read_csv('BrandName.csv')
brand = df['Brand_ENG'].tolist()
print(brand)
brand_name = []
for i in brand:
    i = i.replace(' ','')
    brand_name.append(i)
print(brand_name)

['MUSINSASTANDARD', 'ASICS', 'ADIDAS', 'LMOOD', 'LAFUDGESTORE', 'TOFFEE', 'TRILLION', 'DRAW FIT', 'CODEGRAPHY', 'YALE', 'GLW', 'SATUR', 'DIMITRI BLACK', 'ESPIONAGE', 'LAFUDGE FOR WOMAN', 'DOFFJASON', 'THE NORTH FACE', 'NEW BALANCE', 'FRIZMWORKS', 'OUTSTANDING', 'GAKKAI UNIONS', 'FILLUMINATE', 'TRAVEL', 'XERO', 'MATIN KIM', 'THISISNEVERTHAT', 'SPAO', 'UMBRO', 'WHATITISNT', 'LEE', 'POLO RALPH LAUREN', 'COVERNAT', 'SUARE', 'SOVERMENT', 'UNIFORM BRIDGE', 'NATIONALGEOGRAPHIC', 'PARTIMENTO', '1993STUDIO', 'PUMA', 'DISCOVERY EXPEDITION', 'BRANDED', 'COOR', 'NOT4NERD', 'LIKE THE MOST', 'CPGN STUDIO', 'AVANDRESS', 'INTEMPOMOOD', 'URBANDTYPE', 'KIIMUIR', 'STANDARDERROR', 'JEMUT', 'EZKATON', 'UNION BLUE', 'INSILENCE', 'SPYDER', 'NOMANUAL', 'SCULPTOR', 'CALVIN KLEIN JEANS', 'GLOWNY', 'MLB', 'PLACE STUDIO', 'SIGNATURE', 'HOKA', 'PHYSICAL EDUCATION DEPARTMENT', 'FABREGAT', 'FP142', 'DICKIES', 'MARDI MERCREDI', 'GIORDANO', 'SLOW ACID', 'BROWNBREATH', 'MODNINE', 'IL CORSO', 'TAKEASY', '8SECONDS', 'NON

In [82]:
def save_progress(data, filename='Brand_Info.csv'):
    df = pd.DataFrame(data, columns=['Brand', 'Logo', 'Since', 'Country', 'Info'])
    if not os.path.exists(filename):  
        df.to_csv(filename, index=False)
    else:  
        df.to_csv(filename, mode='a', header=False, index=False)

def load_completed_brands(filename='completed_brands.csv'):
    try:
        return pd.read_csv(filename)['Brand'].tolist()
    except FileNotFoundError:
        return []

def save_completed_brands(brands, filename='completed_brands.csv'):
    pd.DataFrame(brands, columns=['Brand']).to_csv(filename, index=False)

options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage') 
options.add_argument("--no-sandbox") 
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)
time.sleep(1)
completed_brands = load_completed_brands()


for brand in tqdm(brand_name, desc='Brands'):
    if brand in completed_brands:
        continue 
    
    url = f'https://www.musinsa.com/brands/{brand}'
    driver.get(url)

    try:
        logo_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/p[1]/img')
        since_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/p[2]')
        country_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[1]/div/span/img')
        info_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[3]/div[2]/div[3]/div[1]/span[1]')
       
        logo = logo_element.get_attribute('src') 
        since = since_element.text
        country = country_element.get_attribute('alt') 
        info = info_element.text
        
        # 각 브랜드 처리 후 바로 CSV 파일에 저장
        save_progress([[brand, logo, since, country, info]])
        
        # 처리 완료된 브랜드 목록 업데이트
        completed_brands.append(brand)
        save_completed_brands(completed_brands)

    except NoSuchElementException:
        continue

driver.quit()

Brands: 100%|██████████| 3993/3993 [1:26:48<00:00,  1.30s/it]


상품들 가져오기

In [27]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tqdm import tqdm
import time
import pandas as pd
import re
import os

In [29]:
df = pd.read_csv('Brand.csv')
brand = df['Brand_ENG'].tolist()
print(brand)
brand_name = []
for i in brand:
    i = i.replace(' ','')
    brand_name.append(i)
print(brand_name)

['MUSINSASTANDARD', 'LMOOD', 'LAFUDGESTORE', 'TOFFEE', 'TRILLION', 'DRAWFIT', 'CODEGRAPHY', 'SATUR', 'DIMITRIBLACK', 'ESPIONAGE', 'DOFFJASON', 'FRIZMWORKS', 'OUTSTANDING', 'FILLUMINATE', 'TRAVEL', 'XERO', 'MATINKIM', 'THISISNEVERTHAT', 'SPAO', 'COVERNAT', 'SUARE', 'UNIFORMBRIDGE', 'PARTIMENTO', '1993STUDIO', 'DISCOVERYEXPEDITION', 'BRANDED', 'COOR', 'NOT4NERD', 'LIKETHEMOST', 'INTEMPOMOOD', 'URBANDTYPE', 'KIIMUIR', 'STANDARDERROR', 'JEMUT', 'EZKATON', 'UNIONBLUE', 'INSILENCE', 'NOMANUAL', 'SCULPTOR', 'GLOWNY', 'PLACESTUDIO', 'SIGNATURE', 'FABREGAT', 'MARDIMERCREDI', 'SLOWACID', 'BROWNBREATH', 'MODNINE', 'ILCORSO', 'TAKEASY', '8SECONDS', 'NONBLANK', 'XTONZ', '1989STANDARD', 'ROUGHSIDE', 'PERSTEP', 'MINDBRIDGE', 'URBANSTOFF', 'VIVASTUDIO', 'THECOLDESTMOMENT', 'BRENSON', 'NOIRER', 'TYPESERVICE', 'THOMASMORE', 'LEMARD', 'MANINSTORE', 'TWN', 'ROMANTICCROWN', 'BLACKMOMENT', 'WVPROJECT', 'MAISONMINED', 'NICEGHOSTCLUB', 'TOPTEN', 'LETTERCOMO', 'PLAC', 'WKNDRS', 'LADYVOLUME', 'SUPERSUB', 'WHOLO

In [30]:

def save_progress(completed_brands, filename='completed_brands.csv'):
    """진행 상태를 저장하는 함수"""
    pd.DataFrame(completed_brands, columns=['Brand', 'Category']).to_csv(filename, index=False)

def load_progress(filename='completed_brands.csv'):
    """저장된 진행 상태를 로드하는 함수"""
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

# 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)

# 브랜드명 리스트 로드
df = pd.read_csv('Brand.csv')
brand_names = df['Brand_ENG'].tolist()

# 카테고리 목록
categories = ['상의', '아우터', '바지', '원피스', '스커트']

# 이미 완료된 브랜드와 카테고리 로드
completed_brands = load_progress()

data = []

for brand in tqdm(brand_names, desc='Brands'):
    for category in tqdm(categories, desc='Categories', leave=False):
        if {'Brand': brand, 'Category': category} in completed_brands:
            continue
        driver.get(f'https://www.musinsa.com/brands/{brand}')

        # 카테고리 선택
        try:
            category_button = driver.find_element(By.XPATH, f"//a[contains(text(), '{category}')]")
            category_button.click()
            time.sleep(1)  # 페이지 로딩 대기
        except NoSuchElementException:
            print(f"Category {category} not found in brand {brand}")
            continue

        # 현재 카테고리 페이지의 모든 상품 처리
        for item in range(1, 91):  # 한 페이지에 최대 90개 상품이 있다고 가정
            try:
                # 제품 가격 
                item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
                item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                item_price = item_price_element.text  # 가격 정보 텍스트 수집

                # 상품 상세 페이지로 이동
                item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[1]/a')
                driver.execute_script("arguments[0].click();", item_element)
                time.sleep(1)  # 상세 페이지 로딩 대기
                
                # 상품 상세 정보 수집
                item_img = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
                item_name = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
                brand_name_element = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a')
                item_num = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text
                item_season = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text
                item_sex = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text
                
                # 수집된 데이터 추가
                data.append([brand, category, item_img, item_name, brand_name_element.text, item_num, item_season, item_sex, item_price])

                driver.back()  # 상품 목록 페이지로 돌아가기
                time.sleep(1)  # 페이지 로딩 대기
            except NoSuchElementException:
                print(f"Item not found in the list, item number: {item}")
                break  # 상품 목록의 끝에 도달

        # 카테고리가 완료되면 진행 상태를 저장하고 데이터를 저장합니다.
        completed_brands.append({'Brand': brand, 'Category': category})
        save_progress(completed_brands)

        # 수집된 데이터를 중간 저장합니다.
        new_data = pd.DataFrame(data, columns=['Brand', 'Category', 'Item_Image', 'Item_Name', 'Brand_Name', 'Item_Number', 'Item_Season', 'Item_Sex', 'Item_Price'])
        if not os.path.exists('Items_Info.csv'):
            new_data.to_csv('Items_Info.csv', index=False)
        else:
            new_data.to_csv('Items_Info.csv', mode='a', header=False, index=False)
        data.clear()  # 다음 반복을 위해 데이터 리스트를 비웁니다.

driver.quit()


Brands:   0%|          | 0/2929 [00:00<?, ?it/s]

Item not found in the list, item number: 1


Brands:   0%|          | 0/2929 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [32]:

def save_progress(completed_brands, filename='completed_brands.csv'):
    """진행 상태를 저장하는 함수"""
    pd.DataFrame(completed_brands, columns=['Brand', 'Category']).to_csv(filename, index=False)

def load_progress(filename='completed_brands.csv'):
    """저장된 진행 상태를 로드하는 함수"""
    try:
        return pd.read_csv(filename).to_dict('records')
    except FileNotFoundError:
        return []

# 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.set_window_size(1200, 2000)

# 브랜드명 리스트 로드
df = pd.read_csv('Brand.csv')
brand_names = df['Brand_ENG'].tolist()

# 카테고리 목록
categories = ['상의', '아우터', '바지', '원피스', '스커트']

# 이미 완료된 브랜드와 카테고리 로드
completed_brands = load_progress()

data = []

for brand in tqdm(brand_names, desc='Brands'):
    for category in tqdm(categories, desc='Categories', leave=False):
        if {'Brand': brand, 'Category': category} in completed_brands:
            continue
        driver.get(f'https://www.musinsa.com/brands/{brand}')

        # 카테고리 선택
        try:
            category_button = driver.find_element(By.XPATH, f"//a[contains(text(), '{category}')]")
            category_button.click()
            time.sleep(1)  # 페이지 로딩 대기
        except NoSuchElementException:
            print(f"Category {category} not found in brand {brand}")
            continue

        # 현재 카테고리 페이지의 모든 상품 처리
        for item in range(1, 91):  # 한 페이지에 최대 90개 상품이 있다고 가정
            try:
                # 제품 가격 
                item_price_xpath = f'//*[@id="searchList"]/li[{item}]/div[4]/div[2]/p[3]'
                print(item_price_xpath)
                item_price_element = driver.find_element(By.XPATH, item_price_xpath)
                break
                item_price = item_price_element.text  # 가격 정보 텍스트 수집

                # 상품 상세 페이지로 이동
                item_element = driver.find_element(By.XPATH, f'//*[@id="searchList"]/li[{item}]/div[3]/div[1]/a')
                driver.execute_script("arguments[0].click();", item_element)
                time.sleep(1)  # 상세 페이지 로딩 대기
                
                # 상품 상세 정보 수집
                item_img = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[1]/div[1]/div/img').get_attribute('src')
                item_name = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[1]/div[3]/div[2]/h3').text
                brand_name_element = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/a')
                item_num = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[1]/div[2]/span[2]').text
                item_season = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[1]').text
                item_sex = driver.find_element(By.XPATH, '//*[@id="root"]/div[1]/div[2]/div[2]/div[2]/ul/li[2]/div[2]/span[3]').text
                
                # 수집된 데이터 추가
                data.append([brand, category, item_img, item_name, brand_name_element.text, item_num, item_season, item_sex, item_price])

                driver.back()  # 상품 목록 페이지로 돌아가기
                time.sleep(1)  # 페이지 로딩 대기
            except NoSuchElementException:
                print(f"Item not found in the list, item number: {item}")
                break  # 상품 목록의 끝에 도달

        # 카테고리가 완료되면 진행 상태를 저장하고 데이터를 저장합니다.
        completed_brands.append({'Brand': brand, 'Category': category})
        save_progress(completed_brands)

        # 수집된 데이터를 중간 저장합니다.
        new_data = pd.DataFrame(data, columns=['Brand', 'Category', 'Item_Image', 'Item_Name', 'Brand_Name', 'Item_Number', 'Item_Season', 'Item_Sex', 'Item_Price'])
        if not os.path.exists('Items_Info.csv'):
            new_data.to_csv('Items_Info.csv', index=False)
        else:
            new_data.to_csv('Items_Info.csv', mode='a', header=False, index=False)
        data.clear()  # 다음 반복을 위해 데이터 리스트를 비웁니다.

driver.quit()


Brands:   0%|          | 0/2929 [00:00<?, ?it/s]

//*[@id="searchList"]/li[1]/div[4]/div[2]/p[3]


//*[@id="searchList"]/li[1]/div[4]/div[2]/p[3]


//*[@id="searchList"]/li[1]/div[4]/div[2]/p[3]


Brands:   0%|          | 0/2929 [00:19<?, ?it/s]

//*[@id="searchList"]/li[1]/div[4]/div[2]/p[3]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00007FF6C866AD02+56930]
	(No symbol) [0x00007FF6C85DF602]
	(No symbol) [0x00007FF6C84942E5]
	(No symbol) [0x00007FF6C8471D4C]
	(No symbol) [0x00007FF6C85023F7]
	(No symbol) [0x00007FF6C8517891]
	(No symbol) [0x00007FF6C84FBA43]
	(No symbol) [0x00007FF6C84CD438]
	(No symbol) [0x00007FF6C84CE4D1]
	GetHandleVerifier [0x00007FF6C89E6F8D+3711213]
	GetHandleVerifier [0x00007FF6C8A404CD+4077101]
	GetHandleVerifier [0x00007FF6C8A3865F+4044735]
	GetHandleVerifier [0x00007FF6C8709736+706710]
	(No symbol) [0x00007FF6C85EB8DF]
	(No symbol) [0x00007FF6C85E6AC4]
	(No symbol) [0x00007FF6C85E6C1C]
	(No symbol) [0x00007FF6C85D68D4]
	BaseThreadInitThunk [0x00007FF848F1257D+29]
	RtlUserThreadStart [0x00007FF84A88AA58+40]
